# Round Robin Tournament Planner

### Or: Recieving an email titled "In need of data analytics help"

I recently recieved the following email from my in-laws:
>  We have a not hypothetical problem for which we thought data engineers might have an easy solution, so we figured we would consult with you.  We have been having Zoom get togethers with [*Family Members*].  The format we have found to work well is to have breakout sessions with each breakout room having two people who (obviously) are not spouses.  In order to make this work over several sessions, what we need is a list  of all possible combinations, so that we can effectively rotate the combinations without repetition over a few sessions.

>So, there are 5 couples.  Can you determine all of the possible combinations of 2 person breakout rooms where each person is not paired with his or her spouse.

The problem statement reads almost identical to a game tournament where each competitor must play every other competitor with no duplicates, and every competitor plays a game in every round. Indeed, this is known as a [round-robin tournament](https://en.wikipedia.org/wiki/Round-robin_tournament).

A round-robin tournament is often designed using the *circle method*. All competitors are given a numeric id `i` ranging from 1 to the total number of competitors `n`. For the first round, players are paired with competitor id `i+(n-i)`. For example, if there are 10 competitors then competitor 1 is paried with 10, 2 with 9, 3 with 8, and so on:

<p style="text-align: center; font-weight: bold;">Round 1</p>

| Pair A | Pair B  | Pair C  | Pair D  | Pair E  |
|---------|---------|---------|---------|---------|
| 1       | 2       | 3       | 4       | 5       |
| 10      | 9       | 8       | 7       | 6       |

One competitor is chosen as the fixed point (we will choose `1`), and all other competitors are rotated clockwise 1 position around the fixed point.

<p style="text-align: center; font-weight: bold;">Round 2</p>

| Pair A | Pair B  | Pair C  | Pair D  | Pair E  |
|---------|---------|---------|---------|---------|
| *1*      | 10    |   2     | 3       | 4       | 
| 9       | 8       | 7       | 6       | 5 |

<p style="text-align: center; font-weight: bold;">Round 3</p>

| Pair A | Pair B  | Pair C  | Pair D  | Pair E  |
|---------|---------|---------|---------|---------|
| *1*     | 9      | 10        | 2       | 3       |
| 8       | 7       | 6       |5       |4       | 

This repeats for `n-1` rounds until all competitors have played each other!

It was clear my in-laws needed a round-robin planner with a slight modification to account for spouses. Surprsingly I could not easily find one online (that was not related to [round-robin scheduling](https://en.wikipedia.org/wiki/Round-robin_scheduling) ), and so I decided to write a simple round-robin script that can be used again and again - for Zoom family get togethers, or family game tear aparts.

### Round Robin Script

In [1]:
import numpy as np

def create_rounds(n):
    '''Returns a list of lists, where each sublist represents a game round. 
       Each of the n players is paired with another player, with no duplication across rounds'''
    
    # There will be n-1 unique rounds
    r = n - 1

    # Create our player ids! 
    p_ids = np.arange(n)

    # Find the split point and create a reversed player ids array
    split = len(p_ids)//2 - 1
    p_ids_rev = p_ids[:split:-1].copy()

    # We will store each round in a list
    rounds = []

    # First round will be a simple match-up (if odd, 1 player match to themselves)
    pairs = list(zip(p_ids,p_ids_rev))
    rounds.append(pairs)

    # For the remaining rounds, keep first player fixed and execute round robin
    while len(rounds) < r:
        # Insert the first index of the reversed ids next to the "fixed" player
        p_ids = np.insert(p_ids,1,p_ids_rev[0])

        # Subtract 1 from each player id
        p_ids_rev = p_ids_rev-1

        # Reset if it hits 0
        p_ids_rev[p_ids_rev==0] = n-1

        # Match up the pairs
        pairs = list(zip(p_ids, p_ids_rev))
        rounds.append(pairs)
    return rounds

def print_rounds(rounds):
    ''' Prints our rounds with their pairings'''
    for idx, round in enumerate(rounds):
        print(f'Round {idx+1}:',round)

Now that we have our script, we'll apply it to my in-laws' case. They have 10 people from 5 couples.

In [2]:
# How many players? 
n = 10

# Create and print rounds with n players using round robin method
rounds = create_rounds(n)
print_rounds(rounds)

Round 1: [(0, 9), (1, 8), (2, 7), (3, 6), (4, 5)]
Round 2: [(0, 8), (9, 7), (1, 6), (2, 5), (3, 4)]
Round 3: [(0, 7), (8, 6), (9, 5), (1, 4), (2, 3)]
Round 4: [(0, 6), (7, 5), (8, 4), (9, 3), (1, 2)]
Round 5: [(0, 5), (6, 4), (7, 3), (8, 2), (9, 1)]
Round 6: [(0, 4), (5, 3), (6, 2), (7, 1), (8, 9)]
Round 7: [(0, 3), (4, 2), (5, 1), (6, 9), (7, 8)]
Round 8: [(0, 2), (3, 1), (4, 9), (5, 8), (6, 7)]
Round 9: [(0, 1), (2, 9), (3, 8), (4, 7), (5, 6)]


Great! So now we have all possible rounds where each individual will be paired with a person *and* it is not someone they have already been paired with before! But what about spouses? My in-laws stated they they did not want to be paired up with their spouse (which would be an odd method to talk to someone they live with, after all). 

Here we can simply use 1 round as a "Spouse Round" and assign numeric ids to people based on that pairing. That round is then discarded, but the id's are used for the remaining rounds. For example, we will use Round 1 as the "Spouse Round" and label our pairs based on that round. Each letter represents a spouse pair. Round 1 will then be skipped during the Zoom call.

In [3]:
names = {0:'A1', 9: 'A2', 1:'B1', 8:'B2', 2:'C1', 7:'C2', 3:'D1', 6: 'D2', 4:'E1', 5:'E2'}
for idx, round in enumerate(rounds):
    named = [( names[j],names[k] ) for j,k in round]
    print(f'Round {idx+1}:',named)

Round 1: [('A1', 'A2'), ('B1', 'B2'), ('C1', 'C2'), ('D1', 'D2'), ('E1', 'E2')]
Round 2: [('A1', 'B2'), ('A2', 'C2'), ('B1', 'D2'), ('C1', 'E2'), ('D1', 'E1')]
Round 3: [('A1', 'C2'), ('B2', 'D2'), ('A2', 'E2'), ('B1', 'E1'), ('C1', 'D1')]
Round 4: [('A1', 'D2'), ('C2', 'E2'), ('B2', 'E1'), ('A2', 'D1'), ('B1', 'C1')]
Round 5: [('A1', 'E2'), ('D2', 'E1'), ('C2', 'D1'), ('B2', 'C1'), ('A2', 'B1')]
Round 6: [('A1', 'E1'), ('E2', 'D1'), ('D2', 'C1'), ('C2', 'B1'), ('B2', 'A2')]
Round 7: [('A1', 'D1'), ('E1', 'C1'), ('E2', 'B1'), ('D2', 'A2'), ('C2', 'B2')]
Round 8: [('A1', 'C1'), ('D1', 'B1'), ('E1', 'A2'), ('E2', 'B2'), ('D2', 'C2')]
Round 9: [('A1', 'B1'), ('C1', 'A2'), ('D1', 'B2'), ('E1', 'C2'), ('E2', 'D2')]


So we see how this works for an even number of people. What if another family member who does not have a spouse joins the Zoom call?

In [4]:
# How many players? 
n = 11

# Create and print rounds with n players using round robin method
rounds = create_rounds(n)
print_rounds(rounds)

Round 1: [(0, 10), (1, 9), (2, 8), (3, 7), (4, 6), (5, 5)]
Round 2: [(0, 9), (10, 8), (1, 7), (2, 6), (3, 5), (4, 4)]
Round 3: [(0, 8), (9, 7), (10, 6), (1, 5), (2, 4), (3, 3)]
Round 4: [(0, 7), (8, 6), (9, 5), (10, 4), (1, 3), (2, 2)]
Round 5: [(0, 6), (7, 5), (8, 4), (9, 3), (10, 2), (1, 1)]
Round 6: [(0, 5), (6, 4), (7, 3), (8, 2), (9, 1), (10, 10)]
Round 7: [(0, 4), (5, 3), (6, 2), (7, 1), (8, 10), (9, 9)]
Round 8: [(0, 3), (4, 2), (5, 1), (6, 10), (7, 9), (8, 8)]
Round 9: [(0, 2), (3, 1), (4, 10), (5, 9), (6, 8), (7, 7)]
Round 10: [(0, 1), (2, 10), (3, 9), (4, 8), (5, 7), (6, 6)]


Well that's okay, for each round 1 person is paired with themself. Maybe they can use that round to go get a cup of tea.